In [792]:
#Loader function for Pytorch Dataset Folder thingy
def load_data(path: str):
    #print(f"Loading file: {path}")
    tensor = torch.load(path)
    #tensor = tensor.float()
    #print(f"Loaded tensor type: {type(tensor)} {tensor.type()}")
    return tensor


#ERROR DOESNT FULLY WORK WITH DATASET - IDK WHY - I GAVE UP FOR NOW
#FIXME THIS DOES WORK AS A REGULAR REMAP AND I USE THIS FOR PRINTING DATA
#NOTE TO SELF: DO NOT DELETE THIS FUNCTION
# Define a custom transform to convert labels from float to int
def label_remap(label):
    label_mapping = {
            0: 47,
            1: 23,
            2: 27,
            3: 34,
            4: 43,
            5: 5,
            6: 41,
            7: 14,
            8: 46,
            9: 22,
            10: 37,
            11: 38,
            12: 24,
            13: 3,
            14: 12,
            15: 13,
            16: 9,
            17: 20,
            18: 0,
            19: 33,
            20: 30,
            21: 29,
            22: 44,
            23: 48,
            24: 25,
            25: 4,
            26: 39,
            27: 49,
            28: 40,
            29: 6,
            30: 7,
            31: 32,
            32: 26,
            33: 31,
            34: 2,
            35: 17,
            36: 10,
            37: 1,
            38: 11,
            39: 8,
            40: 42,
            41: 21,
            42: 28,
            43: 19,
            44: 18,
            45: 45,
            46: 36,
            47: 35,
            48: 15,
            49: 16
    }
    return torch.tensor(int(label_mapping[label]))


#Summary: Print All Samples in Dataloader
#Summary: Data Label OG = this is the published index labling, dog = 4
#Summary; because pytorch sorts the data, dog = 18 in Dataloader,
#Summary: so if i want to double check that the labels are correct, I have to use label_remap() function to convert the pytorch label number to the original
#Summary: this doesnt really matter, all the code below was to just do a double check that everything is correct

def print_data_loader(_dataloader, _stop,class_names):
    _idx = f"{'#':<{4}} | "
    _dshape = f"{'Data Shape:[c,  t,   w]':<{24}} | "
    _lshape = f"{'Label Shape':<{15}} | "
    _ddtype = f"{'Data DataType':<{13}} | "
    _dlabel1 = f"{'Data Label OG':<{13}} | "
    _dlabel2 = f"{'Data Label Py':<{13}} | "
    _dlabel3 = f"{'Data Label ':<{20}} | "
    _ldtype = f"{'Label Type':<{22}} | "
    _dpath = f"{'Data Path + Label PyTorch Value':<{60}} | "

    print(f"{_idx}{_dshape}{_lshape}{_ddtype}{_dlabel1}{_dlabel2}{_dlabel3}{_dpath}{_ldtype}")

    #Note: CREATE AN ITERATOR FOR DATALOADER
    _data_iter = iter(_dataloader)
    #next(_data_iter)
    counter = 1  #counter when to stop for loop
    #NOTE: GO OVER ALL SAMPLES IN ITERATOR
    for (_data, _label) in _data_iter:
        #_data_iter = iter(test_dataloader)
        _rnd_idx = _data_iter._sampler_iter.gi_frame.f_locals['idx']
        _sample_idx = _data_iter._dataset.indices[_rnd_idx]
        #numm = _data_iter._index_sampler.sampler.data_source.indices[num] #incorrect location
        _path = test_dataset.dataset.samples[_sample_idx]

        #NOTE: This is for debug to check that it is the same samples values
        #_amy_data = my_data_set[NUM2][0]
        #_amy_dataog =  _data[0]
        #diff =sum(sum(_amy_data - _amy_dataog)) 
        #print(diff)
        #print(_path)
        _idx = f"{counter:<{4}} | "
        _dshape = f"{str(_data.shape):<{24}} | "  # = [batch_size, color_channels, height, width]"
        _lshape = f"{str(_label.shape):<{15}} | "
        _ddtype = f"{str(_data.dtype):<{13}} | "
        _dlabel1 = f"{str(label_remap(int(_label[0]))):<{13}} | "
        _dlabel2 = f"{str(_label):<{13}} | "
        _dlabel3 = f"{str(class_names[_label]):<{20}} | "
        _ldtype = f"{str(type(_label)):<{22}} | "
        _dpath = f"{str(_path).split('/', 6)[6]:<{60}} | "

        print(f"{_idx}{_dshape}{_lshape}{_ddtype}{_dlabel1}{_dlabel2}{_dlabel3}{_dpath}{_ldtype}")

        if counter == _stop:
            break
        else:
            counter = counter + 1



# ALL CODE ABOVE IS FUNCTIONS FROM MFCC_DATA_IMPORT

In [793]:
import os
OutputFolder ="/Users/daniel/Desktop/Github/ESC_Dissertation_2024_EEE/MFCC/tensors/"
# Step 1: Create a dataset from the new organized folder

from torchvision import transforms
#data_transform = transforms.Compose([transforms.ToTensor()])
# Define a custom transform to add the extra dimension
test_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.unsqueeze(0))  # Adds an extra dimension, making it [1, C, H, W]
])
import torch
from torchvision import datasets

my_data_set = datasets.DatasetFolder(
        root = OutputFolder,
        loader = load_data,
        transform = test_transform,  #our transformation for data
        target_transform = None,
        extensions = ('.pt',)
)
#my_data_set.class_to_idx
#Step 2
# Determine the sizes for training and testing splits
# And Split the dataset

from torch.utils.data import random_split

train_size = int(0.8 * len(my_data_set))
test_size = len(my_data_set) - train_size

train_dataset, test_dataset = random_split(my_data_set, [train_size, test_size])


In [794]:
#step 3
# Turn train\test Datasets into train\test DataLoaders

#import os
#NUM_OF_WORKERS = os.cpu_count() #this is used on collab when I want to use all max cpu power
from torch.utils.data import DataLoader

BATCH_SIZE = 20
train_dataloader = DataLoader(dataset = train_dataset,
                              batch_size = BATCH_SIZE,  # how many samples per batch?
                              #num_workers=1,#ERROR THIS DOESNT WORK ON LAPTOP BUT WORKS ON COLLAB # how many CPU subprocesses to use for data loading? (higher = more)
                              shuffle = True)  # shuffle the data?

test_dataloader = DataLoader(dataset = test_dataset,
                             batch_size = BATCH_SIZE,  # how many samples per batch?
                             #num_workers=1,#ERROR THIS DOESNT WORK ON LAPTOP BUT WORKS ON COLLAB # how many CPU subprocesses to use for data loading? (higher = more)
                             shuffle = True)  # shuffle the data?

In [795]:
class_names = my_data_set.classes
class_dict = my_data_set.class_to_idx
print(f"'Class Label': Label Number \n{class_dict}")

'Class Label': Label Number 
{'airplane': 0, 'breathing': 1, 'brushing_teeth': 2, 'can_opening': 3, 'car_horn': 4, 'cat': 5, 'chainsaw': 6, 'chirping_birds': 7, 'church_bells': 8, 'clapping': 9, 'clock_alarm': 10, 'clock_tick': 11, 'coughing': 12, 'cow': 13, 'crackling_fire': 14, 'crickets': 15, 'crow': 16, 'crying_baby': 17, 'dog': 18, 'door_wood_creaks': 19, 'door_wood_knock': 20, 'drinking_sipping': 21, 'engine': 22, 'fireworks': 23, 'footsteps': 24, 'frog': 25, 'glass_breaking': 26, 'hand_saw': 27, 'helicopter': 28, 'hen': 29, 'insects': 30, 'keyboard_typing': 31, 'laughing': 32, 'mouse_click': 33, 'pig': 34, 'pouring_water': 35, 'rain': 36, 'rooster': 37, 'sea_waves': 38, 'sheep': 39, 'siren': 40, 'sneezing': 41, 'snoring': 42, 'thunderstorm': 43, 'toilet_flush': 44, 'train': 45, 'vacuum_cleaner': 46, 'washing_machine': 47, 'water_drops': 48, 'wind': 49}


In [796]:
print(f"Train DataLoader Samples:\t{len(train_dataloader)},\t\tTest DataLoader Samples:\t{len(test_dataloader)}.")
print(f"Train DataSet Samples:\t\t{len(train_dataset)},\t\tTest DataSet Samples:\t\t{len(test_dataset)}.")

Train DataLoader Samples:	80,		Test DataLoader Samples:	20.
Train DataSet Samples:		1600,		Test DataSet Samples:		400.


In [797]:
#print_data_loader(test_dataloader, 5,class_names )

In [798]:
#print_data_loader(train_dataloader, 5,class_names )

# CREATING A MODEL USING PYTORCH SEQUENIAL

In [799]:
# Import torch
import torch

# Setup device agnostic code
#device agnostic code - automatically set what cpu\gpu to use for best performance
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Setup random seed
RANDOM_SEED = 42

Using device: mps


In [800]:
import torch
from torch import nn
from torchmetrics import Accuracy

In [801]:

class PiczakRec(nn.Module):
    def __init__(self,inputSize,outputSize,hiddenNeurons,filtersize,poolingsize,padsize,stridesize,flattensize):
        super().__init__()

        self.nnBlock1 = nn.Sequential(
                nn.Conv2d(in_channels=1,out_channels=80,kernel_size=(57,6),padding=0),
                nn.ReLU(),
                #nn.MaxPool2d(kernel_size=(4,3),stride=(1,3)),
        )    
        
        self.nnBlock2 = nn.Sequential(
                nn.Conv2d(in_channels=80,out_channels=80,kernel_size=(1,3),padding=0,stride = 1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(1,3),stride=(1,3))
        )    
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=70720,out_features=5000),
            nn.Linear(in_features=5000,out_features=5000),
            nn.Linear(in_features=5000,out_features=50)
        )
        
    def forward(self, x):
        x = self.nnBlock1(x)
        x = self.nnBlock2(x)
        x = self.classifier(x)
        return x
            

In [802]:
inputSize=1#how many color channels, in my case 1
outputSize=50
hiddenNeurons=1
filtersize=3
poolingsize=2
padsize=0
stridesize=1
flattensize=3528
model = PiczakRec(inputSize,outputSize,hiddenNeurons,filtersize,poolingsize,padsize,stridesize,flattensize)

In [803]:
#model.state_dict()
#next(model.modules())

In [804]:
model.eval()

coeff,y = next(iter(test_dataloader))
model_pred_logits = model(coeff).squeeze(dim=0) # make sure image is right shape + on right device
model_pred_probs = torch.softmax(model_pred_logits, dim=1)
model_pred_label = torch.argmax(model_pred_probs, dim=1)

In [805]:
coeff.shape

torch.Size([20, 1, 498, 13])

# THIS TO TO CALCULATE SIZES - OUTPUT OF LAYERS

In [806]:
from torchshape import tensorshape

op = nn.Conv2d(in_channels=1,out_channels=80,kernel_size=(57,6),padding=0)
outshape = tensorshape(op, in_shape = (1,1,498,13))
print(f"N\tC\tH\tW\t");print(f"{outshape[0]}\t{outshape[1]}\t{outshape[2]}\t{outshape[3]}\t")

N	C	H	W	
1	80	442	8	


In [807]:
op1 = nn.MaxPool2d(kernel_size=(4,3),stride=(1,3))
outshape = tensorshape(op=op1, in_shape = outshape)
print(f"N\tC\tH\tW\t"); print(f"{outshape[0]}\t{outshape[1]}\t{outshape[2]}\t{outshape[3]}\t")

N	C	H	W	
1	80	439	2	


In [808]:
op = nn.Conv2d(in_channels=80,out_channels=80,kernel_size=(1,3),padding=0,stride = 1)
outshape = tensorshape(op, in_shape = outshape)
print(f"N\tC\tH\tW\t");print(f"{outshape[0]}\t{outshape[1]}\t{outshape[2]}\t{outshape[3]}\t")

N	C	H	W	
1	80	439	0	


In [809]:
op1 = nn.MaxPool2d(kernel_size=(1,3),stride=(1,3))
outshape = tensorshape(op=op1, in_shape = outshape)
print(f"N\tC\tH\tW\t");print(f"{outshape[0]}\t{outshape[1]}\t{outshape[2]}\t{outshape[3]}\t")

N	C	H	W	
1	80	439	0	


In [810]:
op2 = nn.Flatten()
outshape = tensorshape(op=op2, in_shape = (1,1,439,1))
print(f"H\tW\t");print(f"{outshape[0]}\t{outshape[1]}")

H	W	
1	439


In [811]:
acc_func = Accuracy(task="multiclass", num_classes=50).to(device)


from tqdm.auto import tqdm 

model = model.to(device)
# Create a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
train_acc = []
test_acc = []
### Training loop
epochs = 100
for epoch in tqdm(range(epochs)):
  train_loss = 0
  for batch, (X, y) in enumerate(tqdm(train_dataloader)):
    model.train()
    # Put data on CPU
    X = X.to(device)
    y = y.to(device)
    # Forward pass
    if torch.isnan(X).any() or torch.isinf(X).any():
        print("Data contains NaN or Inf values.")

    logits = model(X)
    y_pred = torch.softmax(logits,dim=1)#;print(y_pred.shape)
    y_hat = torch.argmax(y_pred,dim=1) #;print(y_hat.shape);print(y_hat)
    
    # Loss calculation
    loss = loss_fn(logits, y)
    
    
    train_acc.append(acc_func(y,y_hat)) #this slows down the runtime
    train_loss += loss
    # Optimizer zero grad
    optimizer.zero_grad()
    # Loss backward
    loss.backward()
    # Step the optimizer
    optimizer.step()
  # Adjust train loss for number of batches
    train_loss /= len(train_dataloader)
    #print(f"Epoch: {epoch} | Loss: {train_loss:.3f} | Acc: {train_acc}") 
  ### Testing loop
    test_loss_total = 0

  # Put model in eval mode 
  model.eval()

  # Turn on inference mode
  with torch.inference_mode():
    for batch, (X_test, y_test) in enumerate(test_dataloader):
      # Make sure test data on CPU
      X_test, y_test = X_test.to(device), y_test.to(device)
      test_pred = model(X_test)

      logits = model(X_test)
      test_loss = loss_fn(logits, y_test)
      y_pred = torch.softmax(logits,dim=1)#;print(y_pred.shape)
      y_hat = torch.argmax(y_pred,dim=1)
      test_acc.append(acc_func(y,y_hat)) #this slows down the runtime

      
      test_loss_total += test_loss
     
    test_loss_total /= len(test_dataloader)

  # Print out what's happening
  print(f"Epoch: {epoch} | Loss: {train_loss:.3f} | Test loss: {test_loss_total:.3f} | Train Acc: {train_acc[epoch]} | Test Acc: {test_acc[epoch]}")


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 0 | Loss: 0.047 | Test loss: 3.997 | Train Acc: 0.0 | Test Acc: 0.0


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 1 | Loss: 0.049 | Test loss: 3.944 | Train Acc: 0.0 | Test Acc: 0.05000000074505806


...

Epoch: 43 | Loss: 0.040 | Test loss: 4.382 | Train Acc: 0.0 | Test Acc: 0.0


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 44 | Loss: 0.035 | Test loss: 4.355 | Train Acc: 0.0 | Test Acc: 0.0


  0%|          | 0/80 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/model1_test_trash')

ModuleNotFoundError: No module named 'tensorboard'

In [2]:
import torch.utils.tensorboard

ModuleNotFoundError: No module named 'tensorboard'